In [1]:
#dataSetTrainer

import cv2,os
import sys
import numpy as np
from PIL import Image

# Get user supplied values
databasePath = #Add the path to your .jpg database (only named by number) 
imagePath = #Add the path to the image you want to be recognized
face_cascadePath = #Add the path to haarcascade_frontalface_default.xml"
eye_cascadePath = #Add the path to haarcascade_eye.xml"
upperbody_cascadePath = #Add the path to haarcascade_upperbody.xml"
fullbody_cascadePath = #Add the path to haarcascade_fullbody.xml"

# Create the haar cascade
faceCascade = cv2.CascadeClassifier(face_cascadePath)
eyeCascade = cv2.CascadeClassifier(eye_cascadePath)
upperbodyCascade = cv2.CascadeClassifier(upperbody_cascadePath)
fullbodyCascade = cv2.CascadeClassifier(fullbody_cascadePath)

recognizer = cv2.face.createLBPHFaceRecognizer()

def getImagesAndLabels(databasePath):
    imagePaths = [os.path.join(databasePath,f) for f in os.listdir(databasePath)] #get the path of all the files in the folder
    faceSamples = [] #create empty face list
    Ids = [] #create empty ID list
    for imagePath in imagePaths: #now looping through all the image paths and loading the Ids and the images
        if(os.path.split(imagePath)[-1].split(".")[-1]!='jpg'): # ignore if the file does not have jpg extension :
            continue
        pilImage = Image.open(imagePath).convert('L') #loading the image and converting it to gray scale
        imageNp = np.array(pilImage,'uint8') #Now we are converting the PIL image into numpy array
        Id = int(os.path.split(imagePath)[-1].split(".")[0]) #getting the Id from the image
        faces = faceCascade.detectMultiScale(imageNp) # extract the face from the training image sample
        for (x,y,w,h) in faces: #If a face is there then append that in the list as well as Id of it
            faceSamples.append(imageNp[y:y+h,x:x+w])
            Ids.append(Id)
    return faceSamples, Ids

faces, Ids = getImagesAndLabels(databasePath)
recognizer.train(faces, np.array(Ids))
recognizer.save() #Add path where you want to save the trainer.yml


In [7]:
#faceRecognizer

import cv2,os
import sys
import numpy as np
from PIL import Image

# Get user supplied values
databasePath = #Add the path to your .jpg database (only named by number) 
imagePath = #Add the path to the image you want to be recognized
face_cascadePath = #Add the path to haarcascade_frontalface_default.xml"
eye_cascadePath = #Add the path to haarcascade_eye.xml"
upperbody_cascadePath = #Add the path to haarcascade_upperbody.xml"
fullbody_cascadePath = #Add the path to haarcascade_fullbody.xml"

# Create the haar cascade
faceCascade = cv2.CascadeClassifier(face_cascadePath)
eyeCascade = cv2.CascadeClassifier(eye_cascadePath)
upperbodyCascade = cv2.CascadeClassifier(upperbody_cascadePath)
fullbodyCascade = cv2.CascadeClassifier(fullbody_cascadePath)

recognizer = cv2.face.createLBPHFaceRecognizer()

recognizer.load() #Add the path to the saved trainer.yml file

# Read the image
image = cv2.imread(imagePath)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
fontFace = cv2.FONT_HERSHEY_TRIPLEX
fontScale = 1
fontColor = (0, 0, 255)

# Detect faces in the image
faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
)

# Draw a rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 51), 1)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = image[y:y+h, x:x+w]
    eyes = eyeCascade.detectMultiScale(roi_gray)
    for (ex, ey, ew, eh) in eyes:
        cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 1)
    upperbod = upperbodyCascade.detectMultiScale(gray)
    for (ux, uy, uw, uh) in upperbod:
        cv2.rectangle(roi_color, (ux, uy), (ux+uw, uy+uh), (255, 0, 0), 1)
    fullbod = fullbodyCascade.detectMultiScale(gray)
    for (fx, fy, fw, fh) in fullbod:
        cv2.rectangle(roi_color, (fx, fy), (fx+fw, fy+fh), (0, 0, 255), 1)
    Id = recognizer.predict(gray[y:y+h,x:x+w])
    #if(Id==17):        #Change the id number and name of person if desired
    #    Id="Obama"
    #else:
    #    Id="Unknown"
    #cv2.putText(image,str(Id), (x,y+h),fontFace, fontScale, fontColor)
    print("Found {0} face(s)!".format(len(faces)))
    print("Found {0} eye(s)!".format(len(eyes)))    
    print("This face belongs to " + Id)
cv2.imshow("Faces found", image)
cv2.waitKey(0)


Found 1 face(s)!
Found 2 eye(s)!
This face belongs to Obama


-1